# 라이브러리 설치 및 데이터 불러오기

In [1]:
from glob import glob
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [2]:
file_names = glob('*.csv')
file_names

['exercise count.csv',
 'finaldata.csv',
 'fulldata_10_31_01_P_golf.csv',
 'fulldata_10_42_01_P_체력단련장업.CSV',
 'number of person.csv',
 'population density.csv',
 '구면적, 인구밀도, 센터밀도.csv',
 '상관계수 구하기.csv',
 '서울시 관광 자연.csv',
 '서울시 주요 공원현황.csv',
 '서울시_연말정산데이터.csv',
 '서울시_인구데이터.csv',
 '서울시사업체수.csv',
 '서울시소득.csv',
 '연령대.csv',
 '인구밀도_20230222102711.csv']

## 첫 번째 데이터 불러오기 및 전처리

In [48]:
p1 = pd.read_csv('서울시 주요 공원현황.csv', encoding='cp949')
p1.head(1)

,연번,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107,450395.554,126.990377,37.55014,http://parks.seoul.go.kr/template/sub/namsan.do


In [36]:
p1.columns

Index(['연번', '공원명', '공원개요', '면적', '개원일', '주요시설', '주요식물', '안내도', '오시는길',
       '이용시참고사항', '이미지', '지역', '공원주소', '관리부서', '전화번호', 'X좌표(GRS80TM)',
       'Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)', '바로가기'],
      dtype='object')

In [49]:
# 불필요한 컬럼 버리기
p_1 = p1.drop(columns=['연번', '공원개요', '개원일', '주요시설', '주요식물', '안내도', '오시는길', '이용시참고사항', '이미지', '관리부서', '전화번호', '바로가기', 'X좌표(GRS80TM)','Y좌표(GRS80TM)'], axis=1)
p_1.head(2)

,공원명,면적,지역,공원주소,X좌표(WGS84),Y좌표(WGS84)
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,중구,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,80683㎡,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394


In [50]:
# '서울특별시' 공원정보만 남기기

p_1 = p_1[p_1['공원주소'].str.contains("서울특별시")]
p_1.head(2)

,공원명,면적,지역,공원주소,X좌표(WGS84),Y좌표(WGS84)
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,중구,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,80683㎡,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394


In [51]:
# 컬럼명 바꾸기

p_1 = p_1.rename(columns={"지역" : "시군구명", "X좌표(WGS84)" : "경도", "Y좌표(WGS84)":"위도"})
p_1.head(2)

,공원명,면적,시군구명,공원주소,경도,위도
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,중구,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,80683㎡,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394


In [52]:
p_1['면적(㎡)'] = p_1['면적'].str.extract(r'(\d+)')
p_1 = p_1.drop(columns=['면적'])

In [54]:
p_1.head(8)

,공원명,시군구명,공원주소,경도,위도,면적(㎡)
0,남산도시자연공원,중구,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140,2896887
1,길동생태공원,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394,80683
3,서울숲,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072,480994
4,월드컵공원,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805,2284085
5,광화문시민열린마당,종로구,서울특별시 종로구 세종로 76-2,126.978023,37.575045,8948
6,진관근린공원(구파발폭포),은평구,서울특별시 은평구 진관동 393,126.919216,37.637024,7
7,중마루근린공원,영등포구,서울특별시 영등포구 영등포동2가 222,126.911267,37.519832,6456
8,응봉공원,성동구,서울특별시 성동구 금호동1가 산1-72,127.021765,37.557228,80309


In [136]:
p_1['분류'] = '공원'
p_1.columns

Index(['공원명', '시군구명', '공원주소', '경도', '위도', '면적(㎡)', '분류'], dtype='object')

In [185]:
p_1.to_csv('seoul_park.csv', index=False)

## 두 번째 데이터 불러오기 및 전처리

In [137]:
p2 = pd.read_csv('서울시 관광 자연.csv', encoding='cp949')
p2.shape

(576, 11)

In [138]:
# 한국어 자료만 추출하기
p_2 = p2[p2['언어'].str.contains('ko')]
p_2.head(2)

,고유번호,언어,상호명,주소,신주소,전화번호,웹사이트,운영시간,운영요일,휴무일,교통정보
0,369,ko,북한산국립공원,136-855 서울 성북구 정릉동 산 1-1,02700 서울 성북구 보국문로34길 6-21 (정릉동),02-909-0497,http://bukhan.knps.or.kr,04:00 ~ 17:00 (3월~11월) 04:00 ~ 16:00 (12월~2월),매일,NaN,"3호선 구파발역 1번 출구로 나와 704번, 34번 버스로 갈아타고 북한산성에서 하..."
3,1981,ko,중랑천 제방,서울 중랑구 면목동 일대,서울특별시 중랑구 면목동 일대,02-938-9520~1,http://jr1000ecocenter.nowon.kr/main.do,NaN,NaN,NaN,"7호선 중화 3번, 4번 출구 도보 10분 1호선 신이문 3번 출구 도보 10분 ..."


In [139]:
# '시군구' 추출해서 컬럼 생성하기
# 끝글자 '구'로 인식하여 인덱싱하기
p_2['시군구명'] = p_2['주소'].str.extract('( \w*구 )')
p_2.head(2)

C:\Users\Myung-ki Min\AppData\Local\Temp\ipykernel_19852\2056628398.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_2['시군구명'] = p_2['주소'].str.extract('( \w*구 )')


,고유번호,언어,상호명,주소,신주소,전화번호,웹사이트,운영시간,운영요일,휴무일,교통정보,시군구명
0,369,ko,북한산국립공원,136-855 서울 성북구 정릉동 산 1-1,02700 서울 성북구 보국문로34길 6-21 (정릉동),02-909-0497,http://bukhan.knps.or.kr,04:00 ~ 17:00 (3월~11월) 04:00 ~ 16:00 (12월~2월),매일,NaN,"3호선 구파발역 1번 출구로 나와 704번, 34번 버스로 갈아타고 북한산성에서 하...",성북구
3,1981,ko,중랑천 제방,서울 중랑구 면목동 일대,서울특별시 중랑구 면목동 일대,02-938-9520~1,http://jr1000ecocenter.nowon.kr/main.do,NaN,NaN,NaN,"7호선 중화 3번, 4번 출구 도보 10분 1호선 신이문 3번 출구 도보 10분 ...",중랑구


In [140]:
p_2.columns

Index(['고유번호', '언어', '상호명', '주소', '신주소', '전화번호', '웹사이트', '운영시간', '운영요일', '휴무일',
       '교통정보', '시군구명'],
      dtype='object')

In [153]:
# 불필요한 컬럼 삭제하기
p2 = p_2.drop(columns=['고유번호', '언어', '주소','전화번호', '웹사이트', '운영시간', '운영요일', '휴무일', '교통정보'])
p2.shape

(118, 3)

In [165]:
# 분류하기 : '산', '공원', '계곡', '길', '섬'&'강'
# 1) 공원부터 추출
pk = p2[p2['상호명'].str.contains("공원")]
pk['분류']  = '공원'
pk.head(2)

C:\Users\Myung-ki Min\AppData\Local\Temp\ipykernel_19852\3148851425.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pk['분류']  = '공원'


,상호명,신주소,시군구명,분류
0,북한산국립공원,02700 서울 성북구 보국문로34길 6-21 (정릉동),성북구,공원
4,구암공원,"07525 서울 강서구 허준로5길 42 (가양동, 구암공원관리사무실)",강서구,공원


In [166]:
# 2) 공원데이터 중복제거
p3 = p2[~p2['상호명'].str.contains("공원")]
# 3) '길' 추출
rd = p3[p3['상호명'].str.contains("길|거리")]
rd['분류'] = '산책로'
rd.shape

C:\Users\Myung-ki Min\AppData\Local\Temp\ipykernel_19852\3505846678.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rd['분류'] = '산책로'


(10, 4)

In [167]:
# 분류하기 : '산', '공원', '계곡', '길', '섬'&'강'
# 4) '공원', '길' 중복제거 데이터 만들기
p4 = p3[~p3['상호명'].str.contains("길")]
# 5) '산' 추출
mt = p4[p4['상호명'].str.contains("산")]
mt['분류'] = '산'
mt.shape

C:\Users\Myung-ki Min\AppData\Local\Temp\ipykernel_19852\2078541372.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mt['분류'] = '산'


(18, 4)

In [168]:
# 나머지 데이터 확인하기
p5 = p4[~p4['상호명'].str.contains("산")]
p5['상호명'].unique()

array(['중랑천 제방', '북서울꿈의숲', '석촌호수 야경', '청계천', '불광천 해담는다리', '노들섬', '금선사',
       '서래섬', '화랑로 낙엽거리', '우이동계곡', '광진숲나루', '춘당지', '홍릉시험림(홍릉숲)',
       '양재시민의 숲', '몽촌토성', '불광천', '홍제천', '뚝섬유원지', '푸른수목원', '백사실계곡',
       '중랑캠핑숲', '노원우주학교', '문화비축기지', '수성동계곡', '녹사평역 지하예술정원', '진관사계곡',
       '서울숲', '양재천&탄천', '한강', '서울식물원', '삼천사계곡'], dtype=object)

In [169]:
#  => '천', '섬', '호수', '계곡', '강'=> 하천
rv = p5[p5['상호명'].str.contains('천|섬|호수|계곡|강')]
rv['분류'] = '하천'
rv.shape

C:\Users\Myung-ki Min\AppData\Local\Temp\ipykernel_19852\3062380553.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv['분류'] = '하천'


(16, 4)

In [170]:
#  나머지값은 '그외'로 분류
p6 = p5[~p5['상호명'].str.contains('천|섬|호수|계곡|강')]
p6['분류'] = '그외'

C:\Users\Myung-ki Min\AppData\Local\Temp\ipykernel_19852\895109025.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p6['분류'] = '그외'


# 녹지정보 값 합치기 + 컬럼명 통일

In [171]:
p_1.columns

Index(['공원명', '시군구명', '공원주소', '경도', '위도', '면적(㎡)', '분류'], dtype='object')

In [176]:
green1 = pd.concat([rd, mt, rv, p6])
green1.head(1)

,상호명,신주소,시군구명,분류
34,망우산 사색의 길,02178 서울 중랑구 망우로 570 (망우묘지관리사무소),중랑구,산책로


In [177]:
# 컬럼명 변경
green1 = green1.rename(columns={'상호명':'공원명', '신주소':'공원주소'})
green1.head(1)

,공원명,공원주소,시군구명,분류
34,망우산 사색의 길,02178 서울 중랑구 망우로 570 (망우묘지관리사무소),중랑구,산책로


In [183]:
# 주소 정리하기 (우편번호 삭제)

''.join(green1['공원주소'].split([0-9],1))

AttributeError: 'Series' object has no attribute 'split'

In [ ]:
# 도로명주소만 남기기


In [163]:
# 합치기

green = pd.concat([pk, rd, mt, rv, p6])